In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [18]:
print("hello")

hello


In [ ]:
import json
tweets = []
for line in open('../input/tweets.json', 'r'):
    tweets.append(json.loads(line))

In [19]:
tweets[0]

{'in_reply_to_status_id_str': None,
 'text': 'If mitt romney say he gunna legalize weed then obama is fucked',
 'geo': None,
 'in_reply_to_screen_name': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'id_str': '246058205367644160',
 'retweeted': False,
 'created_at': 'Thu Sep 13 01:30:22 +0000 2012',
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'entities': {'urls': [], 'user_mentions': [], 'hashtags': []},
 'in_reply_to_status_id': None,
 'place': None,
 'contributors': None,
 'coordinates': None,
 'favorited': False,
 'user': {'follow_request_sent': None,
  'statuses_count': 23815,
  'profile_background_image_url_https': 'https://si0.twimg.com/profile_background_images/643937019/07gmvz818szy9m7tupnt.jpeg',
  'default_profile_image': False,
  'friends_count': 721,
  'profile_sidebar_border_color': '000000',
  'id_str': '237158178',
  'is_translator': False,
  'verified': False,
  'favourites_count': 39,
  'geo

In [ ]:
obama_cnt = 0
romney_cnt = 0
obama_romney_cnt = 0

import pandas as pd
tweets_df = pd.DataFrame(columns=['id','tweet'],index=None)

for i in range(0,len(tweets)):
    obama = False;
    romney = False;
    for word in tweets[i]['text'].split():
#         print(word)
        if word.lower() in ["obama","barack","barackobama","obamabarack"]:
            obama = True
        if word.lower() in["mitt","romney","mittromney","romneymitt"]:
            romney = True
    if obama == True and romney == False:
        data = pd.DataFrame({"id":[tweets[i]['id_str']],"tweet":[tweets[i]['text']]})
        tweets_df = tweets_df.append(data, ignore_index = True)
        obama_cnt += 1
    elif obama == False and romney == True:
        data = pd.DataFrame({"id":[tweets[i]['id_str']],"tweet":[tweets[i]['text']]})
        tweets_df = tweets_df.append(data, ignore_index = True)
        romney_cnt += 1
#     elif obama == True and romney == True:
#         obama_romney_cnt += 1
print(obama_cnt)
print(romney_cnt)
print(obama_romney_cnt)

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix


In [ ]:
import re
processed_tweet = []
for i in range (0,len(tweets_df)):
    if(i % 10000 == 0):
        print("i:",i)
    x = tweets_df.iloc[i]['tweet']
#     tweets_df.iloc[i]['tweet'] = ' '.join(re.sub("(RT)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
    temp = ' '.join(re.sub("(RT)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
    processed_tweet.append(temp)
print(processed_tweet)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer_word = TfidfVectorizer(sublinear_tf=True,
                                       analyzer='word',
                                       stop_words='english', 
                                       token_pattern=r'\w{1,}', 
                                       norm='l2',
                                       ngram_range=(1,2),
                                       max_features=50000)
mat = tfidf_vectorizer_word.fit_transform(processed_tweet)

In [ ]:
mat.shape

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(mat)

In [ ]:
kmeans.labels_

In [ ]:
from collections import Counter
Counter(kmeans.labels_)

In [ ]:
tweets_df['processed_tweet'] = processed_tweet

In [20]:
tweets_df.head()

,id,tweet,processed_tweet
0,246058206651117568,Can we please stop pretending that Obama is a ...,Can we please stop pretending that Obama is a ...
1,246058207460614144,RT @MotherJones: The official GOP response to ...,The official GOP response to Americans being k...
2,246058208463044608,RT @PolarCoug: Obama would never make a good r...,Obama would never make a good running back He ...
3,246058208584671234,RT @Zack_gale: You cant bag on Obama if you on...,gale You cant bag on Obama if you only pay att...
4,246058212116267008,@SethLavin national pressure on both sides to ...,national pressure on both sides to end it will...


In [21]:
is_obama = []
is_romney = []
for i in range(0,len(tweets_df)):
    if(i % 20000 == 0):
        print("i:",i)
    obama = False;
    romney = False;
    for word in tweets_df.iloc[i]['processed_tweet'].split():
#         print(word)
        if word.lower() in ["obama","barack","obamabarack","barackobama"]:
            obama = True
            break
        elif word.lower() in["mitt","romney","mittromney","romneymitt"]:
            romney = True
            break
    is_obama.append(obama)
    is_romney.append(romney)
tweets_df['is_obama'] = is_obama
tweets_df['is_romney'] = is_romney

i: 0
i: 20000
i: 40000
i: 60000
i: 80000
i: 100000
i: 120000
i: 140000
i: 160000
i: 180000
i: 200000
i: 220000
i: 240000
i: 260000
i: 280000
i: 300000
i: 320000
i: 340000


In [24]:
tweets_df.head()

,id,tweet,processed_tweet,is_obama,is_romney,label
0,246058206651117568,Can we please stop pretending that Obama is a ...,Can we please stop pretending that Obama is a ...,True,False,0
1,246058207460614144,RT @MotherJones: The official GOP response to ...,The official GOP response to Americans being k...,True,False,0
2,246058208463044608,RT @PolarCoug: Obama would never make a good r...,Obama would never make a good running back He ...,True,False,0
3,246058208584671234,RT @Zack_gale: You cant bag on Obama if you on...,gale You cant bag on Obama if you only pay att...,True,False,0
4,246058212116267008,@SethLavin national pressure on both sides to ...,national pressure on both sides to end it will...,True,False,0


In [23]:
tweets_df['label'] = kmeans.labels_

In [26]:
obama_pos = 0
obama_neg = 0
obama_neu = 0
romney_pos = 0
romney_neg = 0
romney_neu = 0
for i in range (0, len(tweets_df)):
    if(i % 20000 == 0):
        print("i:",i)
    if tweets_df.iloc[i]['is_obama'] == True and tweets_df.iloc[i]['label'] == 0:
        obama_pos += 1
    elif tweets_df.iloc[i]['is_obama'] == True and tweets_df.iloc[i]['label'] == 1:
        obama_neg += 1
    elif tweets_df.iloc[i]['is_romney'] == True and tweets_df.iloc[i]['label'] == 0:
        romney_pos += 1
    elif tweets_df.iloc[i]['is_romney'] == True and tweets_df.iloc[i]['label'] == 1:
        romney_neg += 1
        
print("obama_pos",obama_pos)
print("obama_neg",obama_neg)
print("romney_pos",romney_pos)
print("romney_neg",romney_neg)


i: 0
i: 20000
i: 40000
i: 60000
i: 80000
i: 100000
i: 120000
i: 140000
i: 160000
i: 180000
i: 200000
i: 220000
i: 240000
i: 260000
i: 280000
i: 300000
i: 320000
i: 340000
obama_pos 306989
obama_neg 18808
romney_pos 29097
romney_neg 3
